In [1]:
# List all device
from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [2]:
# Check available GPU
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";  

In [4]:
# Importing the libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Reshape, Lambda
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.activations import softmax
from keras.optimizers import SGD
import math
import pickle

In [ ]:
with open("../data/dowjones_calculated/periods.txt", "rb") as fp:   # Unpickling
    dataset = pickle.load(fp)

In [ ]:
returns = dataset[0][0][0]
returns = (returns - returns.mean()) / returns.std()
X_train = returns.values
y_train = dataset[0][0][1].values

# returns = dataset[1][0][0]
# returns = (returns - returns.mean()) / returns.std()
X_test = dataset[1][0][0].values
y_test = dataset[1][0][1].values

In [ ]:
def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])
y_train = get_one_hot(y_train, 2)
y_test = get_one_hot(y_test, 2)

In [ ]:
print(f"x train shape: {X_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"x test shape: {X_test.shape}")
print(f"y test shape: {y_test.shape}")
# print(f"predicted shape: {predicted.shape}")

x train shape: (750, 31)
y train shape: (750, 31, 2)
x test shape: (490, 31)
y test shape: (490, 31, 2)


In [ ]:
data = X_train
targets = y_train

train_gen = TimeseriesGenerator(data, targets,
                               length=240, sampling_rate=1,
                               batch_size=510)

In [ ]:
data = X_test
targets = y_test

test_gen = TimeseriesGenerator(data, targets,
                               length=240, sampling_rate=1,
                               batch_size=250)

In [ ]:
X_train = train_gen[0][0]
y_train = train_gen[0][1]
X_test = test_gen[0][0]
y_test = test_gen[0][1]

In [ ]:
print(f"x train shape: {X_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"x test shape: {X_test.shape}")
print(f"y test shape: {y_test.shape}")

x train shape: (510, 240, 31)
y train shape: (510, 31, 2)
x test shape: (250, 240, 31)
y test shape: (250, 31, 2)


In [ ]:
X_train = X_train.transpose((0,2,1))
X_train = np.reshape(X_train, (510 * 31, 240))
y_train = np.reshape(y_train, (510 * 31, 2))

In [ ]:
# Reshaping X_train for efficient modelling
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
# X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
print(f"x train shape: {X_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"x test shape: {X_test.shape}")
print(f"y test shape: {y_test.shape}")

x train shape: (15810, 240, 1)
y train shape: (15810, 2)
x test shape: (250, 240, 31)
y test shape: (250, 31, 2)


In [ ]:
# expected input data shape: (batch_size, timesteps, data_dim)

# The LSTM architecture
regressor = Sequential()
# First LSTM layer with Dropout regularisation
# regressor.add(LSTM(units=25, return_sequences=True, input_shape=(240, 31)))
regressor.add(LSTM(units=25, input_shape=(240, 1), dropout=0.1))

# The output layer
# regressor.add(LSTM(25))
regressor.add(Dense(2, activation='softmax'))
# regressor.add(Reshape((31,2)))
# regressor.add(softmax(axis = -1))
# regressor.add(Lambda(lambda  x: softmax(x, axis = -1)))
# Compiling the RNN
regressor.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
# regressor.compile(loss='mean_squared_error',
#               optimizer='rmsprop',
#               metrics=['accuracy'])
# Fitting to the training set
regressor.fit(X_train,y_train,epochs=100,batch_size=32, validation_split=0.1, callbacks = [EarlyStopping(monitor='val_acc', patience=50),
             ModelCheckpoint(filepath='../model/LSTM/best_model.h5', monitor='val_acc', save_best_only=True)])

Train on 14229 samples, validate on 1581 samples
Epoch 1/100
14229/14229 [==============================] - 81s 6ms/step - loss: 0.6929 - acc: 0.5119 - val_loss: 0.6925 - val_acc: 0.5225
Epoch 2/100
14229/14229 [==============================] - 80s 6ms/step - loss: 0.6928 - acc: 0.5150 - val_loss: 0.6926 - val_acc: 0.5149
Epoch 3/100
14229/14229 [==============================] - 80s 6ms/step - loss: 0.6927 - acc: 0.5135 - val_loss: 0.6932 - val_acc: 0.5168
Epoch 4/100
14229/14229 [==============================] - 80s 6ms/step - loss: 0.6926 - acc: 0.5135 - val_loss: 0.6924 - val_acc: 0.5275
Epoch 5/100
 5216/14229 [=========>....................] - ETA: 48s - loss: 0.6925 - acc: 0.5197

In [ ]:
# Evaluating our model
predicted = regressor.predict(X_test)

In [ ]:
label = predicted > 0.5
label = label * 1 # Convert boolean to int

In [ ]:
(sum(y_test[:, :, 1] == label[:, :, 1])/(y_test.size/2)).sum()

In [ ]:
model_name = '../model/LSTM/my_model3.h5'
regressor.save(model_name)  # creates a HDF5 file 'my_model.h5'
del regressor  # deletes the existing model

In [ ]:
from keras.models import load_model
from keras.activations import softmax

# returns a compiled model
# identical to the previous one
regressor1 = load_model(model_name, custom_objects={"softmax": softmax})

In [ ]:
predicted_load = regressor1.predict(X_test)
label_load = predicted_load > 0.5
label_load = label_load * 1 # Convert boolean to int
(sum(y_test[:, :, 1] == label_load[:, :, 1])/(y_test.size/2)).sum()